## MBenes

MAB phases possess variable chemical formulas with the relevant compositions being MAB, M2AB2, M3AB4, and M4AB6, for which M represents an early transition metal and A stands for an IIIA and IVA group element.[6] When assuming A = Al for the sake of simplicity, MAlB phases exhibit an orthorhombic crystal structure with space groups varying from Cmcm (for MAlB) over Cmmm (for M2AlB2 and M4AlB6) to Pmmm (for M3AlB4).[6]

For MBenes, it is important to emphasize that the M-B layers are separated by either mono- and bi-layers of Al.

The crystal structures of various MAB phases and related MBenes (Figure 2a) can be classified in the following manner. The first group of compounds are “212” phases with Cmmm crystal symmetry, “414” phases with Immm symmetry, and “222” with Cmcm symmetry.[3] The second group consists of “314” structures with Pmmm symmetry, whereas the third group is formed by “416” structures with Cmmm symmetry.



<div align="left">
<img src="../figures/fig_001.png" width = "40%" />
</div>

In [24]:
from ase import Atom, Atoms
from ase.build import mx2, molecule
from ase.data import chemical_symbols
from ase.io import read, write
from ase.spacegroup import get_spacegroup
from ase.visualize import view
from itertools import combinations
import numpy as np
import random

class MBeneBuilder:
    """
    用于搭建高对称的单过渡金属 MBene 2D结构的 python 类。
    """
    def __init__(self, structure_type="MBene", prototype="212_Cmmm", m_element="Cr", a_element="Al", b_element="B", 
                 lattice_a=0.0, lattice_b=0.0, lattice_c=0.0, supercell_matrix=(3, 3, 1), vacuum=15.0, n_layers=1, layer_distance=5.0, **kwargs):
        self.structure_type = structure_type
        self.prototype = prototype
        self.m_element = m_element
        self.a_element = a_element
        self.b_element = b_element
        self.lattice_a = lattice_a
        self.lattice_b = lattice_b
        self.lattice_c = lattice_c
        self.supercell_matrix = supercell_matrix
        self.vacuum = vacuum
        self.n_layers = n_layers
        self.layer_distance = layer_distance
        
        if self.supercell_matrix[2] != 1:
            print(f"Warning: The third component of supercell_matrix should be 1, but got {self.supercell_matrix[2]}. It will be set to 1.")
            self.supercell_matrix = (self.supercell_matrix[0], self.supercell_matrix[1], 1)
        
    def predefined_systems(self):
        predefined_dict = {
            "212_Cmmm": {
                "symbols": [self.m_element] * 4 + [self.a_element] * 2 + [self.b_element] * 4,
                "cell": [2.908355, 2.909313, 10.985405, 90, 90, 90],
                "scaled_positions": [
                    [0.500000, 0.500000, 0.851860],
                    [0.000000, 0.500000, 0.648140],
                    [0.000000, 0.500000, 0.351860],
                    [0.500000, 0.500000, 0.148140],
                    [0.000000, 0.000000, 0.000000],
                    [0.500000, 0.000000, 0.500000],
                    [0.500000, 0.000000, 0.705053],
                    [0.000000, 0.000000, 0.794947],
                    [0.000000, 0.000000, 0.205053],
                    [0.500000, 0.000000, 0.294947],
                ],
                "removed_index": [0,1,4,5,6,7],
            },
            "222_Cmcm": {
                "symbols": [self.m_element] * 4 + [self.a_element] * 4 + [self.b_element] * 4,
                "cell": [3.205721, 3.096498, 13.969478, 90, 90, 90],
                "scaled_positions": [
                    [0.500000, 0.250000, 0.410469],
                    [0.000000, 0.750000, 0.089531],
                    [0.000000, 0.250000, 0.910469],
                    [0.500000, 0.750000, 0.589531],
                    [0.000000, 0.750000, 0.301286],
                    [0.500000, 0.250000, 0.198714],
                    [0.500000, 0.750000, 0.801286],
                    [0.000000, 0.250000, 0.698714],
                    [0.500000, 0.250000, 0.033641],
                    [0.000000, 0.750000, 0.466359],
                    [0.000000, 0.250000, 0.533641],
                    [0.500000, 0.750000, 0.966359],
                ],
                "removed_index": [1,2,4,5,6,7,8,11],
            },
            "314_Pmm2": {
                "symbols": [self.m_element] * 6 + [self.a_element] * 2 + [self.b_element] * 8,
                "cell": [2.918412, 2.920923, 16.116550, 90, 90, 90],
                "scaled_positions": [
                    [0.000000, 0.000000, 0.351004],
                    [0.000000, 0.000000, 0.148996],
                    [0.500000, 0.000000, 0.000000],
                    [0.000000, 0.000000, 0.851004],
                    [0.000000, 0.000000, 0.648996],
                    [0.500000, 0.000000, 0.500000],
                    [0.500000, 0.500000, 0.250000],
                    [0.500000, 0.500000, 0.750000],
                    [0.000000, 0.500000, 0.054005],
                    [0.500000, 0.500000, 0.111895],
                    [0.000000, 0.500000, 0.445996],
                    [0.500000, 0.500000, 0.388105],
                    [0.000000, 0.500000, 0.554005],
                    [0.500000, 0.500000, 0.611895],
                    [0.000000, 0.500000, 0.945996],
                    [0.500000, 0.500000, 0.888105],
                ],
                "removed_index": [1,2,3,6,7,8,9,14,15],
            },
            "322_Cmmm": {
                "symbols": [self.m_element] * 6 + [self.a_element] * 4 + [self.b_element] * 4,
                "cell": [3.038952, 2.942621, 17.826332, 90, 90, 90],
                "scaled_positions": [
                    [0.500000, 0.000000, 0.325048],
                    [0.000000, 0.000000, 0.000000],
                    [0.000000, 0.000000, 0.174952],
                    [0.000000, 0.000000, 0.825048],
                    [0.500000, 0.000000, 0.500000],
                    [0.500000, 0.000000, 0.674952],
                    [0.500000, 0.500000, 0.079035],
                    [0.000000, 0.500000, 0.420965],
                    [0.000000, 0.500000, 0.579035],
                    [0.500000, 0.500000, 0.920965],
                    [0.500000, 0.500000, 0.222358],
                    [0.000000, 0.500000, 0.277642],
                    [0.000000, 0.500000, 0.722358],
                    [0.500000, 0.500000, 0.777642],
                ],
                "removed_index": [1,3,4,5,6,7,8,9,12,13],
            },
            "414_Immm": {
                "symbols": [self.m_element] * 8 + [self.a_element] * 2 + [self.b_element] * 8,
                "cell": [2.934300, 2.973300, 18.891100, 90, 90, 90],
                "scaled_positions": [
                    [0.500000, 0.500000, 0.206400],
                    [0.000000, 0.000000, 0.293600],
                    [0.500000, 0.000000, 0.414100],
                    [0.000000, 0.500000, 0.085900],
                    [0.000000, 0.000000, 0.706400],
                    [0.500000, 0.500000, 0.793600],
                    [0.000000, 0.500000, 0.914100],
                    [0.500000, 0.000000, 0.585900],
                    [0.500000, 0.000000, 0.000000],
                    [0.000000, 0.500000, 0.500000],
                    [0.500000, 0.000000, 0.116100],
                    [0.000000, 0.500000, 0.383900],
                    [0.500000, 0.500000, 0.335400],
                    [0.000000, 0.000000, 0.164600],
                    [0.000000, 0.500000, 0.616100],
                    [0.500000, 0.000000, 0.883900],
                    [0.000000, 0.000000, 0.835400],
                    [0.500000, 0.500000, 0.664600],
                ],
                "removed_index": [4,5,6,7,8,9,14,15,16,17],
            },
            "416_Cmmm": {
                "symbols": [self.m_element] * 8 + [self.a_element] * 2 + [self.b_element] * 12,
                "cell": [2.951706, 3.012993, 21.280000, 90, 90, 90],
                "scaled_positions": [
                    [0.000000, 0.500000, 0.576340],
                    [0.500000, 0.500000, 0.923660],
                    [0.500000, 0.500000, 0.688430],
                    [0.000000, 0.500000, 0.811570],
                    [0.500000, 0.500000, 0.076340],
                    [0.000000, 0.500000, 0.423660],
                    [0.000000, 0.500000, 0.188430],
                    [0.500000, 0.500000, 0.311570],
                    [0.000000, 0.000000, 0.000000],
                    [0.500000, 0.000000, 0.500000],
                    [0.500000, 0.000000, 0.605000],
                    [0.000000, 0.000000, 0.895000],
                    [0.000000, 0.000000, 0.648100],
                    [0.500000, 0.000000, 0.851900],
                    [0.000000, 0.000000, 0.729800],
                    [0.500000, 0.000000, 0.770200],
                    [0.000000, 0.000000, 0.105000],
                    [0.500000, 0.000000, 0.395000],
                    [0.500000, 0.000000, 0.148100],
                    [0.000000, 0.000000, 0.351900],
                    [0.500000, 0.000000, 0.229800],
                    [0.000000, 0.000000, 0.270200],
                ],
                "removed_index": [0,1,2,3,8,9,10,11,12,13,14,15],
            },
            "211_R3m": {
                "symbols": [self.m_element] * 6 + [self.a_element] * 3 + [self.b_element] * 3,
                "cell": [3.496999, 3.496999, 22.076000, 90, 90, 120],
                "scaled_positions": [
                    [0.333300, 0.666700, 0.049600],
                    [0.999967, 0.000033, 0.382933],
                    [0.666633, 0.333367, 0.716267],
                    [0.000000, 0.000000, 0.161200],
                    [0.666667, 0.333333, 0.494533],
                    [0.333333, 0.666667, 0.827867],
                    [0.666700, 0.333300, 0.273000],
                    [0.333367, 0.666633, 0.606333],
                    [0.000033, 0.999967, 0.939667],
                    [0.666700, 0.333300, 0.104400],
                    [0.333367, 0.666633, 0.437733],
                    [0.000033, 0.999967, 0.771067],
                ],
                "removed_index": [0,2,3,5,6,7,8,9,11],
            },
            "211_P63mmc": {
                "symbols": [self.m_element] * 4 + [self.a_element] * 2 + [self.b_element] * 2,
                "cell": [3.523000, 3.523000, 14.489000, 90, 90, 120],
                "scaled_positions": [
                    [0.333300, 0.666700, 0.084400],  
                    [0.666700, 0.333300, 0.584400], 
                    [0.666700, 0.333300, 0.915600],  
                    [0.333300, 0.666700, 0.415600], 
                    [0.666700, 0.333300, 0.250000],  
                    [0.333300, 0.666700, 0.750000],  
                    [0.000000, 0.000000, 0.000000],  
                    [0.000000, 0.000000, 0.500000],  
                ],
                "removed_index": [0,2,4,5,6],
            },
            "211_P-6m2": {
                "symbols": [self.m_element] * 4 + [self.a_element] * 2 + [self.b_element] * 2,
                "cell": [3.503000, 3.503000, 14.700000, 90, 90, 120],
                "scaled_positions": [
                    [0.666700, 0.333300, 0.332000],  
                    [0.666700, 0.333300, 0.668000], 
                    [0.333300, 0.666700, 0.164500],
                    [0.333300, 0.666700, 0.835500],            
                    [0.666700, 0.333300, 0.000000], 
                    [0.000000, 0.000000, 0.500000], 
                    [0.000000, 0.000000, 0.246100],
                    [0.000000, 0.000000, 0.753900], 
                ],
                "removed_index": [1,3,4,5,7],
            },
            "212_P-6m2": {
                "symbols": [self.m_element] * 4 + [self.a_element] * 2 + [self.b_element] * 4,
                "cell": [3.200000, 3.200000, 16.716000, 90, 90, 120],
                "scaled_positions": [
                    [0.333300, 0.666700, 0.349450],  
                    [0.333300, 0.666700, 0.150550],  
                    [0.333300, 0.666700, 0.849450],  
                    [0.333300, 0.666700, 0.650550],  
                    [0.666700, 0.333300, 0.000000],  
                    [0.666700, 0.333300, 0.500000],  
                    [0.000000, 0.000000, 0.250000], 
                    [0.666700, 0.333300, 0.250000], 
                    [0.000000, 0.000000, 0.750000],  
                    [0.666700, 0.333300, 0.750000],  
                ],
                "removed_index": [2,3,4,5,8,9],
            },
            "314_P-6m2": {
                "symbols": [self.m_element] * 6 + [self.a_element] * 2 + [self.b_element] * 8,
                "cell": [3.182000, 3.182000, 23.736000, 90, 90, 120],
                "scaled_positions": [
                    [0.333300, 0.666700, 0.143750], 
                    [0.333300, 0.666700, 0.356250],
                    [0.333300, 0.666700, 0.000000], 
                    [0.333300, 0.666700, 0.643750],
                    [0.333300, 0.666700, 0.856250], 
                    [0.333300, 0.666700, 0.500000],
                    [0.666700, 0.333300, 0.250000], 
                    [0.666700, 0.333300, 0.750000], 
                    [0.666700, 0.333300, 0.073350], 
                    [0.666700, 0.333300, 0.426650],
                    [0.000000, 0.000000, 0.073700], 
                    [0.000000, 0.000000, 0.426300],
                    [0.666700, 0.333300, 0.573350],
                    [0.666700, 0.333300, 0.926650], 
                    [0.000000, 0.000000, 0.573700],
                    [0.000000, 0.000000, 0.926300], 
                ],
                "removed_index": [0,2,4,6,7,8,10,13,15],
            },
        }
        return predefined_dict
    
    def _generate_mab_structure(self):
        print(self.predefined_systems().keys)
        mab_dict = self.predefined_systems()[self.prototype]
        cell = [
            self.lattice_a if self.lattice_a != 0 else mab_dict["cell"][0],
            self.lattice_b if self.lattice_b != 0 else mab_dict["cell"][1],
            self.lattice_c if self.lattice_c != 0 else mab_dict["cell"][2],
            mab_dict["cell"][3],
            mab_dict["cell"][4],
            mab_dict["cell"][5],
        ]
        return Atoms(symbols=mab_dict["symbols"], scaled_positions=mab_dict["scaled_positions"], cell=cell, pbc=True)
    
    def _generate_single_mbene_structure(self):
        mab_dict = self.predefined_systems()[self.prototype]
        structure = self._generate_mab_structure().copy()        
        removed_indices = sorted(mab_dict["removed_index"], reverse=True)
        for index in removed_indices:
            del structure[index]
        structure.center()
        return structure
    
    def _generate_mbene_structure(self):
        structure = self._generate_single_mbene_structure().copy() * self.supercell_matrix

        if self.n_layers > 1:
            single_layer_height = max(atom.position[2] for atom in structure) - min(atom.position[2] for atom in structure)
            for layer in range(1, self.n_layers):
                new_layer = structure.copy()
                for atom in new_layer:
                    atom.position[2] += layer * (single_layer_height + self.layer_distance)
                structure += new_layer
                
        z_length_with_vacuum = (max(atom.position[2] for atom in structure) - min(atom.position[2] for atom in structure)) + self.vacuum
        structure.set_cell([structure.cell[0], structure.cell[1], [0, 0, z_length_with_vacuum]])
        structure.center()
        
        return structure
    
    def build(self):
        if self.structure_type == "MAB":
            structure = self._generate_mab_structure().copy()
        elif self.structure_type == "MBene":
            structure = self._generate_mbene_structure().copy()
        return structure

In [25]:
def identify_system_symmetry(atoms):
    symmetry = get_spacegroup(atoms, symprec=1e-5)
    international_symbol = symmetry.symbol
    spacegroup_number = symmetry.no
    return international_symbol, spacegroup_number
    
system = MBeneBuilder(structure_type="MBene", prototype="414_Immm")
atoms = system.build()
print(identify_system_symmetry(atoms))
view(atoms)
# write("mbene.vasp", atoms, direct=True, sort=True, vasp5=True)

<built-in method keys of dict object at 0x00000262FE3E0480>


<Popen: returncode: None args: ['C:\\Users\\wangchangrui\\.conda\\envs\\Q-Ap...>

### Q-Api 接口

In [ ]:
def mbene_builder(**kwargs):
    kwargs['supercell_matrix'] = (kwargs['n_a'], kwargs['n_b'], 1)
    system = MBeneBuilder(**kwargs)
    new_structure = system.build()
    cif_output = io.BytesIO()
    write(cif_output, new_structure, format='cif')
    cif_string = cif_output.getvalue().decode('utf-8')
    return cif_string

In [ ]:
@guide_register_func(ModelTag('构建高对称 MBene 材料结构', ['催化','电池'],['B'],nano_type='2d').identifier)
def MBene(data):
    mode = data.get('mode')
    if mode == 'init':
        title = Description(name='title', note='构建高对称 MBene 材料结构，需要指定 MAB 相基体元素及晶格参数等信息')
        structure_type = SingleFromList(name='structure_type', 
                                        note='输出 MAB 还是 MBene', 
                                        id='structure_type', 
                                        list_value=[{'label': 'MAB', 'value': 'MAB'},
                                                    {'label': 'MBene', 'value': 'MBene'}], 
                                        default_value='MBene', 
                                        is_required=0)
        prototype = SingleFromList(name='prototype', 
                                   note='MAB 相类型', 
                                   id='prototype', 
                                   list_value=[{'label': i, 'value': i} for i in ['212_Cmmm', '222_Cmcm', '314_Pmm2', '322_Cmmm', '414_Immm', '416_Cmmm', 
                                                                                  '211_R3m', '211_P63mmc', '211_P-6m2', '212_P-6m2', '314_P-6m2']], 
                                   default_value='212_P-6m2', 
                                   is_required=0)
        m_element = SingleFromList(name='m_element', 
                                   note='MAB 相基体中的 M 元素', 
                                   id='m_element', 
                                   list_value=[{'label': '钪', 'value': 'Sc'},
                                               {'label': '钇', 'value': 'Y' },
                                               {'label': '钆', 'value': 'Gd' },
                                               {'label': '钛', 'value': 'Ti'},
                                               {'label': '锆', 'value': 'Zr'},
                                               {'label': '铪', 'value': 'Hf'},
                                               {'label': '钒', 'value': 'V' },
                                               {'label': '铌', 'value': 'Nb'},
                                               {'label': '钽', 'value': 'Ta'},
                                               {'label': '铬', 'value': 'Cr'},
                                               {'label': '钼', 'value': 'Mo'},
                                               {'label': '钨', 'value': 'W' },
                                               {'label': '锰', 'value': 'Mn' },
                                               {'label': '锝', 'value': 'Tc' },
                                               {'label': '铁', 'value': 'Fe' },
                                               {'label': '钴', 'value': 'Co' },
                                               {'label': '钌', 'value': 'Ru' },
                                               {'label': '铑', 'value': 'Rh' },
                                               {'label': '镍', 'value': 'Ni' }],
                                   is_required=0)
        a_element = SingleFromList(name='a_element', 
                                   note='MAB 相基体中的 A 元素', 
                                   id='a_element', 
                                   list_value=[{'label': '铝', 'value': 'Al'},
                                               {'label': '硅', 'value': 'Si'},
                                               {'label': '镓', 'value': 'Ga'},
                                               {'label': '锗', 'value': 'Ge'},
                                               {'label': '铟', 'value': 'In'},
                                               {'label': '铅', 'value': 'Pb'},
                                               {'label': '铊', 'value': 'Tl'},
                                               {'label': '镉', 'value': 'Cd'},
                                               {'label': '铋', 'value': 'Bi'},
                                               {'label': '磷', 'value': 'P'},
                                               {'label': '锡', 'value': 'Sn'}],
                                   is_required=0)
        b_element = SingleFromList(name='b_element', 
                                   note='MAB 相基体中的 B 元素', 
                                   id='b_element', 
                                   list_value=[{'label': '硼', 'value': 'B'}],
                                   is_required=0)
        lattice_a = SingleInput(name='lattice_a', note='a 方向晶格常数 (Å)', id='lattice_a', input_type='float', default_value=0.0, min=0.0, max=100.0, is_required=0)
        lattice_b = SingleInput(name='lattice_b', note='b 方向晶格常数 (Å)', id='lattice_b', input_type='float', default_value=0.0, min=0.0, max=100.0, is_required=0)
        lattice_c = SingleInput(name='lattice_c', note='c 方向晶格常数 (Å)', id='lattice_c', input_type='float', default_value=0.0, min=0.0, max=100.0, is_required=0)
        n_a = SingleInput(name='n_a', note='a 轴的单胞重复单元', id='n_a', input_type='int', default_value=3, min=1, max=10, is_required=0)
        n_b = SingleInput(name='n_b', note='b 轴的单胞重复单元', id='n_b', input_type='int', default_value=3, min=1, max=10, is_required=0)
        vacuum = SingleInput(name='vacuum', note='真空层厚度 (Å)', id='vacuum', input_type='float', default_value=15.0, min=10.0, max=100.0, is_required=0)
        n_layers = SingleInput(name='n_layers', note='MBene 重复单元数', id='n_layers', input_type='int', default_value=1, min=1, max=10, is_required=0)
        layer_distance = SingleInput(name='layer_distance', note='MBene 层间距 (Å)', id='layer_distance', input_type='float', default_value=5.0, min=3.0, max=100.0, is_required=0)
        
        return [title(), structure_type(), prototype(), m_element(), a_element(), b_element(), lattice_a(), lattice_b(), lattice_c(), n_a(), n_b(), vacuum(), n_layers(), layer_distance()]
        
    elif mode == 'generate':
        value = data.get('value')
        try:
            cif_string = mbene_builder(**value)
        except Exception as e:
            return {'error':str(e)}
        
        return {'file_content': cif_string, 'file_format': 'cif'}
